In [1]:
import pandas as pd

In [2]:
def intro():
    """This function creates a starting message which ilustrates the legend
    of further symbols used in the code"""
    print('+---------------------------------+')
    print('|  STRING FROM GRAMMAR GENERATOR  |')
    print('|          G = (N,Ʃ,P,S)          |')
    print('|                                 |')
    print('| N - nonterminal symbols         |')
    print('| Ʃ - terminal symbols            |')
    print('| P - production rules            |')
    print('| S - start symbol                |')
    print('+---------------------------------+')

In [15]:
def read():
    """This function reads all data from the grammer.txt file, which includes
    noterminal symbols, terminal symbols, production rules and the string
    to analyze"""
    global terminal_symbols
    global nonterminal_symbols
    global rules
    global to_analyze_string
    
    file = open('grammar.txt', 'r')

    line = file.readline()
    terminal_symbols = []
    while line[:-1] != 'next':
        terminal_symbols.append(line[:-1])
        line = file.readline()

    line = file.readline()
    nonterminal_symbols = []
    while line[:-1] != 'next':
        nonterminal_symbols.append(line[:-1])
        line = file.readline()

    line = file.readline()
    rules = []
    while line[:-1] != 'next':
        rules.append(line[:-1])
        line = file.readline()
    
    to_analyze_string = file.readline()
    to_analyze_string = to_analyze_string[:-1]
    
    file.close()

In [4]:
def output_intro_data():
    """This function print the read data. Usefull function to check
    that everything is allright."""

    global terminal_symbols
    global nonterminal_symbols
    global rules 
    global to_analyze_string

    print('The grammer is:')
    print('G(N,Ʃ,P,S)')

    output = 'Ʃ = {'
    for x in terminal_symbols:
        output += x + ', '
    output = output[:(len(output) - 2)]
    output += '}'
    print(output)

    output = 'N = {'
    for x in nonterminal_symbols:
        output += x + ', '
    output = output[:(len(output) - 2)]
    output += '}'
    print(output)

    output = 'P = {\n'
    for x in rules:
        output += '\t' + x + '\n'
    output += '}'
    print(output)
    
    print('String to analyze:', to_analyze_string)

In [5]:
def grammar_to_dictionary():
    """This function convers string read production rules in dictionary
    where the key is the left symbol and on right is a list of symbol from 
    the right of arrow, For exeample: A -> ab, {'A' : [['a', 'b']]}"""
    
    global rules_dic
    
    rules_dic = {}
    for rule in rules:
        if not (rule[0] in rules_dic):
            rules_dic[rule[0]] = []
        start = rule.index('>') + 2
        right_production_rule = rule[start:len(rule)]
        symbols = []
        for symbol in right_production_rule:
            symbols.append(symbol)
        rules_dic[rule[0]].append(symbols)
    print(rules_dic)

In [45]:
def create_first_last_table():
    """This function created the the sets FIRST and LAST. Also it prints it.
    The data is stored in first-last dictionary"""
    
    global rules_dic
    global nonterminal_symbols
    global first_last
    
    first_last = {'index' : [], 'FIRST' : [], 'LAST' : []}
    index = []
    first = []
    last = []
    for symbol in rules_dic:
        index.append(symbol) 
        to_search = []
        to_search.append(symbol)
        searched = []
        symbol_first = []
        symbol_last = []
        nr = 0
        while nr < len(to_search):
            if to_search[nr] in rules_dic:
                searched.append(to_search[nr])
                for rule in rules_dic[to_search[nr]]:
                    if rule[0] not in symbol_first:
                        symbol_first.append(rule[0])
                    if rule[0] in nonterminal_symbols and rule[0] not in searched:
                        to_search.append(rule[0])
            nr += 1
        first.append(symbol_first)
        to_search = []
        to_search.append(symbol)
        searched = []
        nr = 0
        while nr < len(to_search):
            if to_search[nr] in rules_dic:
                searched.append(to_search[nr])
                for rule in rules_dic[to_search[nr]]:
                    if rule[-1] not in symbol_last:
                        symbol_last.append(rule[-1])
                    if rule[-1] in nonterminal_symbols and rule[-1] not in searched:
                        to_search.append(rule[-1])
            nr += 1
        last.append(symbol_last)
    first_last['index'] = index
    first_last['FIRST'] = first
    first_last['LAST'] = last
    
    first_last_table = pd.DataFrame(first_last)
    first_last_table.set_index('index')
    
    #print(first_last)
    print(first_last_table)

In [46]:
def construct_matrix():
    """This function creates the matrix of simple precedence, using data
    made earlier in the code."""
    global first_last
    global nonterminal_symbols
    global terminal_symbols
    global rules_dic
    global matrix
    global all_symbols
    
    # 1. Create 2D matrix
    
    # a. Construct a dict with symbol and its nr. order 
    nr = 0
    all_symbols = {}
    for symbol in nonterminal_symbols:
        all_symbols[symbol] = nr
        nr += 1
    for symbol in terminal_symbols:
        all_symbols[symbol] = nr
        nr += 1
    #print(all_symbols, '\n')
    # b. Create the 2D array
    matrix = [[[] for x in range(nr + 1)] for y in range(nr + 1)]
    nr = 1
    for symbol in nonterminal_symbols:
        matrix[0][nr].append(symbol)
        matrix[nr][0].append(symbol)
        nr += 1
    for symbol in terminal_symbols:
        matrix[0][nr].append(symbol)
        matrix[nr][0].append(symbol)
        nr += 1
    #First rule : equal sign (=). Navigate in every production rule, 2 by 2 symbols
    for symbol in rules_dic:
        for rule in rules_dic[symbol]:
            if len(rule) > 1:
                for index in range(len(rule) - 1):
                    symbol_1 = rule[index]
                    symbol_2 = rule[index + 1]
                    if '=' not in matrix[all_symbols[symbol_1] + 1][all_symbols[symbol_2] + 1]:
                        matrix[all_symbols[symbol_1] + 1][all_symbols[symbol_2] + 1].append('=')
    
    #print(pd.DataFrame(matrix))

    #Second rule : smaller sign (<). Navigate in every production rule, 2 by 2
    for symbol in rules_dic:
        for rule in rules_dic[symbol]:
            if len(rule) > 1:
                for index in range(len(rule) - 1):
                    symbol_1 = rule[index]
                    symbol_2 = rule[index + 1]
                    if symbol_2 in nonterminal_symbols:
                        index_symbol_2 = first_last['index'].index(symbol_2)
                        for _symbol in first_last['FIRST'][index_symbol_2]:
                            if '<' not in matrix[all_symbols[symbol_1] + 1][all_symbols[_symbol] + 1]:
                                matrix[all_symbols[symbol_1] + 1][all_symbols[_symbol] + 1].append('<')
    
    #print(pd.DataFrame(matrix))

    #Third rule : bigger sign (>). Navigate in every production rule, 2 by 2
    for symbol in rules_dic:
        for rule in rules_dic[symbol]:
            if len(rule) > 1:
                for index in range(len(rule) - 1):
                    symbol_1 = rule[index]
                    symbol_2 = rule[index + 1]
                    # If symbol_2 is a terminal symbol
                    if symbol_2 in terminal_symbols and symbol_1 in nonterminal_symbols:
                        index_symbol_1 = first_last['index'].index(symbol_1)
                        for _symbol in first_last['LAST'][index_symbol_1]:
                            if '>' not in matrix[all_symbols[_symbol] + 1][all_symbols[symbol_2] + 1]:
                                matrix[all_symbols[_symbol] + 1][all_symbols[symbol_2] + 1].append('>')
                    #If symbol_1 is a nonterminal symbol
                    if symbol_2 in nonterminal_symbols and symbol_1 in nonterminal_symbols:
                        index_symbol_1 = first_last['index'].index(symbol_1)
                        index_symbol_2 = first_last['index'].index(symbol_2)
                        for _symbol_1 in first_last['LAST'][index_symbol_1]:
                            for _symbol_2 in first_last['FIRST'][index_symbol_2]:
                                if _symbol_2 in terminal_symbols:
                                    if '>' not in matrix[all_symbols[_symbol_1] + 1][all_symbols[_symbol_2] + 1]:
                                        matrix[all_symbols[_symbol_1] + 1][all_symbols[_symbol_2] + 1].append('>')
    print(pd.DataFrame(matrix)) 
    

In [54]:
def string_analyses():
    """This function analyze the string given as input. It prints all
    the steps made in the process of analysis."""
    global to_analyze_string
    global matrix
    global all_symbols
    global rules_dic
    
    # Create Analyze list
    analyze_list = []
    analyze_list.append(to_analyze_string[0])
    for index in range(1, len(to_analyze_string)):
        line_position = all_symbols[to_analyze_string[index - 1]] + 1
        column_position = all_symbols[to_analyze_string[index]] + 1
        analyze_list.append(matrix[line_position][column_position][0])
        analyze_list.append(to_analyze_string[index])

    print('Create Analyze list :',''.join(analyze_list))
    
    while len(analyze_list) != 1 and analyze_list[0] != 'S':
        
        # Find production rules which matches sublist in Analyze list
        start_index = 0
        end_index = 1
        while 1:
            if end_index < len(analyze_list) and analyze_list[end_index] != '<' and analyze_list[end_index] != '>':
                end_index += 1
            else:
                potential_rule = analyze_list[start_index:end_index]
                potential_rule = ''.join(potential_rule)
                potential_rule = potential_rule.replace('=','')

                is_potential_rule_ok = False

                for symbol in rules_dic:
                    for rule in rules_dic[symbol]:
                        existing_rule = ''.join(rule)
                        if existing_rule == potential_rule:
                            is_potential_rule_ok = True
                            to_change_symbol = symbol

                if is_potential_rule_ok == True:
                    analyze_list[start_index] = to_change_symbol
                    if end_index < len(analyze_list):
                        analyze_list[end_index] = ' '
                    if start_index > 0:
                        analyze_list[start_index - 1] = ' '
                    del analyze_list[start_index + 1 : end_index]
                    break
                else:
                    start_index = end_index + 1
                    end_index += 2
        print('Change production rule :',''.join(analyze_list))
        
        #Update Analyze list '<', '>', '=' symbols
        for index in range(len(analyze_list)//2):
            line_position = all_symbols[analyze_list[index * 2]] + 1
            column_position = all_symbols[analyze_list[index * 2 + 2]] + 1
            analyze_list[index * 2 + 1] = matrix[line_position][column_position][0]
        print('Update Analyze list :', ''.join(analyze_list))
        
    if len(analyze_list) == 1 and analyze_list[0] == 'S':
        print('Correct!')

In [56]:
"""This is the main code which call all the function to do the
the relations of simple precedence correctly"""
terminal_symbols = []
nonterminal_symbols = []
rules = []
rules_dic = {}
first_last = {}
to_analyze_string = ''
matrix = []
all_symbols = {}

intro()
read()
output_intro_data()
print('_____________________________________________________________________________')
print('Transfering production rules in dictionary. The keys are nonterminal symbols \n')
grammar_to_dictionary()
print('_____________________________________________________________________________')
print('Creating the FIRST-LAST table for Simple Precedence \n')
create_first_last_table()
print('_____________________________________________________________________________')
print('Constructing the matrix of Simple Precedence \n')
construct_matrix()
print('_____________________________________________________________________________')
print('Analyses of ', to_analyze_string, '\n')
string_analyses()

+---------------------------------+
|  STRING FROM GRAMMAR GENERATOR  |
|          G = (N,Ʃ,P,S)          |
|                                 |
| N - nonterminal symbols         |
| Ʃ - terminal symbols            |
| P - production rules            |
| S - start symbol                |
+---------------------------------+
The grammer is:
G(N,Ʃ,P,S)
Ʃ = {a, b, c, d, e, f}
N = {S, A, B, C}
P = {
	S -> Bc
	S -> BcdC
	C -> Ae
	A -> f
	A -> Abf
	B -> a
	B -> Bba
}
String to analyze: abacdfbfe
_____________________________________________________________________________
Transfering production rules in dictionary. The keys are nonterminal symbols 

{'S': [['B', 'c'], ['B', 'c', 'd', 'C']], 'C': [['A', 'e']], 'A': [['f'], ['A', 'b', 'f']], 'B': [['a'], ['B', 'b', 'a']]}
_____________________________________________________________________________
Creating the FIRST-LAST table for Simple Precedence 

  index   FIRST       LAST
0     S  [B, a]  [c, C, e]
1     C  [A, f]        [e]
2     A  [f, A